In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_table(PATH["w.tsv"], index_col=0)

w.columns.name = "Factor"

h = pd.read_table(PATH["h.tsv"], index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
w_signature = pd.DataFrame(index=w.index)

for factor_name, factor_values in w.items():

    w_signature.loc[
        ccal.select_series_indices(
            factor_values,
            ">",
            standard_deviation=1,
            title=dict(
                text="NMF K={} W {} Signature".format(SETTING["nmf_k"], factor_name)
            ),
            xaxis=dict(title="Rank"),
            yaxis=dict(title="Factor Value"),
        ),
        factor_name,
    ] = 1

w_signature.columns = pd.Index(
    ("{} Signature".format(factor_name) for factor_name in w_signature.columns),
    name="Signature",
)

w_signature

In [ ]:
h_signature = pd.DataFrame(columns=h.columns)

for factor_name, factor_values in h.iterrows():

    h_signature.loc[
        factor_name,
        ccal.select_series_indices(
            factor_values,
            ">",
            standard_deviation=1,
            title=dict(
                text="NMF K={} H {} Signature".format(SETTING["nmf_k"], factor_name)
            ),
            xaxis=dict(title="Rank"),
            yaxis=dict(title="Factor Value"),
        ),
    ] = 1

h_signature.index = pd.Index(
    ("{} Signature".format(factor_name) for factor_name in h_signature.index),
    name="Signature",
)

h_signature

In [ ]:
w_element_cluster = pd.read_table(PATH["w|cluster_x_column.tsv"], index_col=0).apply(
    lambda column: column.argmax()
)

w_element_cluster.name = "Cluster"

w_element_cluster.index.name = w.index.name

h_element_cluster = pd.read_table(PATH["h|cluster_x_column.tsv"], index_col=0).apply(
    lambda column: column.argmax()
)

h_element_cluster.name = "Cluster"

h_element_cluster.index.name = h.columns.name

In [ ]:
feature_summary = pd.concat((w, w_signature, w_element_cluster), axis=1)

feature_summary.to_csv(
    "{}/feature_x_summary_information.tsv".format(PATH["summary/"]), sep="\t"
)

feature_summary

In [ ]:
sample_summary = pd.concat((h.T, h_signature.T, h_element_cluster), axis=1)

sample_summary.to_csv(
    "{}/sample_x_summary_information.tsv".format(PATH["summary/"]), sep="\t"
)

sample_summary